# 1 - Genome Wide Association Study

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


import math
import sys
import os

## Introduction

During this step of the workflow we will compute the SNP-PD associations, using imaging information to improve the phenotypic associations.

During the Individual View step, we will focus on finding candidate SNP biomarkers by integrating genetic and neuroimaging data. During this phase, we search for SNP-disease associations accounting for the phenotypic information carried by each imaging measure separately, and by integrating the single results obtained, in order to retrieve comprehensive potential SNP biomarkers for each imaging type considered. 

Thus, we will merge the results obtained using DaTSCAN features in a single results dataset, and those retrieved by using MRI data in another results summary.

To combine the results previously obtained, we use TATES (Van der Sluis *et al*., 2013), which combines the *P*-values in a single-trait-based statistical significance, by correcting for correlation among the imaging features, and evaluating if at least one of the analyzed traits could be associated with a SNP.

In [5]:
genotyping_path = "../../data/genotyping/"
imaging_path = "../../data/imaging/"
patdocs_path = "../../data/patient_docs/"
datscan_path = os.path.join(imaging_path, "DaTSCAN")
mri_path = os.path.join(imaging_path, "MRI")

## GWAS with DaTSCAN features

### Exploring normalized DaTSCAN data

During the previous steps of the workflow we normalized the DaTSCAN data with rank based inverse normal transformation (r-INT).

As previously discussed, we normalized DaTSCAN data because the measurements fell in different range of values. To normalize our data we applied r-INT, in order to provide a normalization procedure less susceptible to outlier values.

In [4]:
datscan = pd.read_csv(
    os.path.join(datscan_path, "DATScan_Analysis_eu_fv_norm.csv")
)
datscan.head(n=10)

,PATNO,EVENT_ID,SCAN_DATE,CAUDATE_R,CAUDATE_L,PUTAMEN_R,PUTAMEN_L
0,3000,SC,2011-01-20,0.932467,1.368655,1.786562,1.448689
1,3001,U01,2011-06-23,-0.313764,-0.507784,-0.313764,-0.648172
2,3002,U01,2011-06-28,0.835925,1.656152,0.032656,0.696547
3,3004,U01,2011-08-24,2.971827,2.971827,2.383049,1.882275
4,3006,SC,2011-08-02,0.038596,-0.188099,-0.408846,-2.499599
5,3008,SC,2011-08-30,1.957186,1.407545,0.941676,1.023443
6,3011,SC,2011-07-07,1.600299,1.848464,2.095619,1.611072
7,3012,SC,2011-10-11,-0.221442,-0.276558,-2.021570,-0.434802
8,3016,SC,2012-01-17,1.510772,1.558962,1.611072,1.262883
9,3018,SC,2012-02-29,-0.360897,-1.018445,-0.823362,-1.730834


### Constructing Phenotype and Covariate files

Before finding statistically significant SNP-phenotype associations with PLINK we need to define two files which are used by the tool to compute the linear model, used during SNP-trait associations:

- phenotype file

- covariate file

The phenotype file contains the phenotypic information we want to add to the model, in our study we add subjects enrolment category (HC or PD) and the imaging feature values.

The covariate file contains all the confounders used to adjust the linear model, in ourstudy we used as covariates the subjects' age, gender and the first 10 principal components of the SNP relatedness matrix.

Let's begin by creating the phenotype file.

In [23]:
pheno_fn = os.path.join(genotyping_path, "phenotype")

# load the subjects enrolment status and demographics informations
status = pd.read_csv(
    os.path.join(patdocs_path, "Patient_Status.csv")
)

# load the IDs of subjects considered in our study
eu_pats_ds = pd.read_csv(
    os.path.join(genotyping_path, "PPMI_eu_woswedd_ds.fam"),
    header=None,
    sep=" "
)

# retrieve status info and demographics for our subjects
status_eu_ds = status[status["PATNO"].isin(eu_pats_ds.iloc[:,0].tolist())]
status_eu_ds.reset_index(inplace=True, drop=True)
status_eu_ds.head(n=10)

,PATNO,RECRUITMENT_CAT,IMAGING_CAT,ENROLL_DATE,ENROLL_CAT,ENROLL_STATUS,DESCRP_CAT,STATUS_DATE
0,3000,HC,HC,02/2011,HC,Enrolled,NaN,02/2011
1,3001,PD,PD,03/2011,PD,Enrolled,NaN,03/2011
2,3002,PD,PD,03/2011,PD,Enrolled,NaN,03/2011
3,3004,HC,HC,04/2011,HC,Enrolled,NaN,04/2011
4,3006,PD,PD,05/2011,PD,Withdrew,NaN,10/2013
5,3008,HC,HC,06/2011,HC,Enrolled,NaN,06/2011
6,3011,HC,HC,07/2011,HC,Withdrew,NaN,07/2014
7,3012,PD,PD,11/2011,PD,Enrolled,NaN,11/2011
8,3016,HC,HC,02/2012,HC,Enrolled,NaN,02/2012
9,3018,PD,PD,04/2012,PD,Enrolled,NaN,04/2012


In [41]:
phenotype = status_eu_ds.merge(datscan, on=["PATNO"])
# extract the columns of interest
phenotype = phenotype.loc[:,["PATNO", "ENROLL_CAT", "CAUDATE_R", "CAUDATE_L", "PUTAMEN_R", "PUTAMEN_L"]]
phenotype.head(n=10)

,PATNO,ENROLL_CAT,CAUDATE_R,CAUDATE_L,PUTAMEN_R,PUTAMEN_L
0,3000,HC,0.932467,1.368655,1.786562,1.448689
1,3001,PD,-0.313764,-0.507784,-0.313764,-0.648172
2,3002,PD,0.835925,1.656152,0.032656,0.696547
3,3004,HC,2.971827,2.971827,2.383049,1.882275
4,3006,PD,0.038596,-0.188099,-0.408846,-2.499599
5,3008,HC,1.957186,1.407545,0.941676,1.023443
6,3011,HC,1.600299,1.848464,2.095619,1.611072
7,3012,PD,-0.221442,-0.276558,-2.021570,-0.434802
8,3016,HC,1.510772,1.558962,1.611072,1.262883
9,3018,PD,-0.360897,-1.018445,-0.823362,-1.730834


We can write the phenotype files. Fpr each imaging feature we create a TXT file containing subjects FID, IID, enrollment category and the DaTSCAN uptake values for the considered feature.

In [43]:
datscan_features = ["CAUDATE_R", "CAUDATE_L", "PUTAMEN_R", "PUTAMEN_L"]
for feature in datscan_features:
    phenotype.loc[:,["PATNO", "PATNO", "ENROLL_CAT", feature]].to_csv(
        os.path.join("".join([pheno_fn, "_", feature, ".txt"])),
        header=False,
        index=False,
        sep=" "
    )

Let's now create the covariate file.

In [51]:
demo = pd.read_csv(
    os.path.join(patdocs_path, "Screening___Demographics.csv")
)
demo = demo[demo["PATNO"].isin(eu_pats_ds.iloc[:,0].tolist())].sort_values(["PATNO"])
demo.reset_index(inplace=True, drop=True)
demo.head(n=10)

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,SIGNCNST,CONSNTDT,APPRDX,CURRENT_APPRDX,P3GRP,...,PRJENRDT,REFERRAL,DECLINED,RSNDEC,EXCLUDED,RSNEXC,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,269535301,V,3000,CONSENT,SCREEN,1.0,01/2011,2.0,2.0,NaN,...,02/2011,31.0,NaN,NaN,NaN,NaN,01/2011,2011-01-19 15:44:58.0,NaN,01/2011
1,274783501,V,3001,CONSENT,SCREEN,1.0,02/2011,1.0,1.0,NaN,...,03/2011,60.0,NaN,NaN,NaN,NaN,02/2011,2011-02-10 13:43:30.0,NaN,03/2011
2,278717701,S,3002,CONSENT,SCREEN,1.0,03/2011,1.0,1.0,NaN,...,03/2011,1.0,NaN,NaN,NaN,NaN,03/2011,2020-04-01 17:33:42.0,NaN,03/2011
3,281159801,V,3004,CONSENT,SCREEN,1.0,03/2011,2.0,2.0,NaN,...,04/2011,99.0,NaN,NaN,NaN,NaN,03/2011,2011-03-30 14:43:13.0,NaN,03/2011
4,283722401,S,3006,CONSENT,SCREEN,1.0,03/2011,1.0,1.0,NaN,...,04/2011,1.0,NaN,NaN,NaN,NaN,03/2011,2020-04-22 09:14:39.0,NaN,03/2011
5,289807201,V,3008,CONSENT,SCREEN,1.0,05/2011,2.0,2.0,NaN,...,06/2011,4.0,NaN,NaN,NaN,NaN,05/2011,2013-10-31 14:28:09.0,NaN,05/2011
6,302607101,V,3011,CONSENT,SCREEN,1.0,06/2011,2.0,2.0,NaN,...,07/2011,1.0,NaN,NaN,NaN,NaN,06/2011,2011-07-22 16:39:28.0,NaN,06/2011
7,316309601,V,3012,CONSENT,SCREEN,1.0,10/2011,1.0,1.0,NaN,...,11/2011,1.0,NaN,NaN,NaN,NaN,10/2011,2011-10-12 13:48:43.0,NaN,10/2011
8,329167601,V,3016,CONSENT,SCREEN,1.0,01/2012,2.0,2.0,NaN,...,02/2012,4.0,NaN,NaN,NaN,NaN,01/2012,2012-01-12 13:19:24.0,NaN,01/2012
9,336629801,V,3018,CONSENT,SCREEN,1.0,02/2012,1.0,1.0,NaN,...,03/2012,60.0,NaN,NaN,NaN,NaN,03/2012,2012-05-17 12:12:53.0,NaN,03/2012


Let's now compute the subjects age at the DaTSCAN visit time.

In [59]:
def compute_age(df):
    scan_year = int(df["SCAN_DATE"].split("-")[0])
    birth_year = int(df["BIRTHDT"])
    return (scan_year - birth_year)

x = demo.merge(datscan, on=["PATNO"])
demo["AGE"] = x.apply(compute_age, axis=1)
demo.loc[:, ["PATNO", "AGE"]].head(n=10)  # display subjects age

,PATNO,AGE
0,3000,70
1,3001,65
2,3002,68
3,3004,60
4,3006,58
5,3008,82
6,3011,32
7,3012,58
8,3016,58
9,3018,61


Since we removed three subjects from those of step 2 of preprocessing phase we must recompute the PCs of the relatedness matrix in order to have consistent data with our current dataset.

As we did before, to compute the first 20 PCs we use PLINK PCA.

In [18]:
ppmi_eu_woswedd_ds_fn = os.path.join(genotyping_path, "PPMI_eu_woswedd_ds")

!plink --bfile {ppmi_eu_woswedd_ds_fn} --pca 20 --out {ppmi_eu_woswedd_ds_fn}

PLINK v1.90b6.12 64-bit (28 Oct 2019)          www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ../../data/genotyping/PPMI_eu_woswedd_ds.log.
Options in effect:
  --bfile ../../data/genotyping/PPMI_eu_woswedd_ds
  --out ../../data/genotyping/PPMI_eu_woswedd_ds
  --pca 20

16384 MB RAM detected; reserving 8192 MB for main workspace.
128812 variants loaded from .bim file.
422 people (286 males, 136 females) loaded from .fam.
Using up to 15 threads (change this with --threads).
Before main variant filters, 422 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.996652.
128812 variants and 422 people pass filters and QC.
Note: No phenotypes present.
Relationship matrix calculation complete.


In [20]:
ppmi_pca = pd.read_csv(".".join([ppmi_eu_woswedd_ds_fn, "eigenvec"]), sep=' ', header=None)

# assign FID and IID
cnames = ['FID','IID']
for i in range(1, 21):
    cnames.append("PC"+str(i))
    
ppmi_pca.columns = cnames
ppmi_pca.head(n=10)

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20
0,3000,3000,0.023104,0.001445,0.038672,0.065804,-0.007233,-0.032171,0.041329,0.020136,...,0.006711,0.038946,0.035519,0.011342,0.045500,0.099110,-0.092471,-0.061807,0.035368,0.053554
1,3001,3001,0.015770,0.014693,-0.003507,-0.048157,-0.043188,-0.063182,0.035549,-0.039994,...,0.044206,-0.014498,-0.015049,-0.024854,-0.049211,0.032961,-0.047794,0.025384,-0.064395,-0.009126
2,3002,3002,-0.082304,-0.035871,-0.021713,0.012172,0.028592,0.001123,-0.010880,-0.001130,...,-0.027085,0.010237,-0.010473,-0.079551,0.055008,0.058719,0.037635,0.031731,-0.082463,-0.020639
3,3004,3004,0.023939,0.032296,-0.007026,-0.002677,-0.049808,-0.047968,0.072133,0.032667,...,-0.040209,0.027052,0.050784,0.055534,0.093572,0.027401,0.033852,-0.096813,-0.046014,-0.011014
4,3006,3006,0.004569,0.059148,0.101874,-0.011916,-0.001887,-0.047472,-0.001574,-0.023827,...,-0.004358,0.020863,0.022857,-0.004821,-0.017148,0.013909,-0.035708,-0.088609,-0.034012,-0.001674
5,3008,3008,0.013419,-0.003194,-0.049605,0.049230,-0.048325,0.045633,-0.042648,-0.003362,...,-0.000836,-0.126883,-0.051652,0.123715,-0.005408,-0.006208,-0.008207,-0.028819,-0.049820,-0.130877
6,3011,3011,-0.045717,0.024360,0.029712,0.028696,0.027011,-0.002224,-0.007908,0.000338,...,-0.014150,-0.027093,-0.024393,0.062394,-0.010077,-0.001595,-0.008321,-0.038271,0.027300,-0.005139
7,3012,3012,-0.006437,-0.015327,0.014593,-0.004998,-0.072540,0.063095,-0.023632,0.024530,...,0.014611,-0.027948,-0.029460,0.006143,-0.015118,-0.039220,0.006339,0.019902,0.009799,0.032107
8,3016,3016,0.039748,0.001799,-0.084670,0.028990,0.043786,-0.000456,-0.016404,-0.028868,...,0.089533,-0.086117,-0.060762,-0.038292,0.039845,-0.073360,0.022928,0.034004,-0.010412,-0.056705
9,3018,3018,0.016829,0.001896,0.067707,-0.080170,0.063767,0.062882,0.020254,-0.058311,...,0.016343,0.060883,-0.033268,0.099699,-0.080082,-0.032918,0.050229,-0.007243,-0.075981,0.017879


Let's now merge PC values to demographic data. 

In [65]:
covariate_fn = os.path.join(genotyping_path, "covariate.txt")

covariate = ppmi_pca.merge(demo, left_on=["IID"], right_on=["PATNO"])
covariate["GENDER"] = eu_pats_ds[4]  # solve 0s in demographics
covariate.head(n=10)

,FID,IID,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,REFERRAL,DECLINED,RSNDEC,EXCLUDED,RSNEXC,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV,AGE
0,3000,3000,0.023104,0.001445,0.038672,0.065804,-0.007233,-0.032171,0.041329,0.020136,...,31.0,NaN,NaN,NaN,NaN,01/2011,2011-01-19 15:44:58.0,NaN,01/2011,70
1,3001,3001,0.015770,0.014693,-0.003507,-0.048157,-0.043188,-0.063182,0.035549,-0.039994,...,60.0,NaN,NaN,NaN,NaN,02/2011,2011-02-10 13:43:30.0,NaN,03/2011,65
2,3002,3002,-0.082304,-0.035871,-0.021713,0.012172,0.028592,0.001123,-0.010880,-0.001130,...,1.0,NaN,NaN,NaN,NaN,03/2011,2020-04-01 17:33:42.0,NaN,03/2011,68
3,3004,3004,0.023939,0.032296,-0.007026,-0.002677,-0.049808,-0.047968,0.072133,0.032667,...,99.0,NaN,NaN,NaN,NaN,03/2011,2011-03-30 14:43:13.0,NaN,03/2011,60
4,3006,3006,0.004569,0.059148,0.101874,-0.011916,-0.001887,-0.047472,-0.001574,-0.023827,...,1.0,NaN,NaN,NaN,NaN,03/2011,2020-04-22 09:14:39.0,NaN,03/2011,58
5,3008,3008,0.013419,-0.003194,-0.049605,0.049230,-0.048325,0.045633,-0.042648,-0.003362,...,4.0,NaN,NaN,NaN,NaN,05/2011,2013-10-31 14:28:09.0,NaN,05/2011,82
6,3011,3011,-0.045717,0.024360,0.029712,0.028696,0.027011,-0.002224,-0.007908,0.000338,...,1.0,NaN,NaN,NaN,NaN,06/2011,2011-07-22 16:39:28.0,NaN,06/2011,32
7,3012,3012,-0.006437,-0.015327,0.014593,-0.004998,-0.072540,0.063095,-0.023632,0.024530,...,1.0,NaN,NaN,NaN,NaN,10/2011,2011-10-12 13:48:43.0,NaN,10/2011,58
8,3016,3016,0.039748,0.001799,-0.084670,0.028990,0.043786,-0.000456,-0.016404,-0.028868,...,4.0,NaN,NaN,NaN,NaN,01/2012,2012-01-12 13:19:24.0,NaN,01/2012,58
9,3018,3018,0.016829,0.001896,0.067707,-0.080170,0.063767,0.062882,0.020254,-0.058311,...,60.0,NaN,NaN,NaN,NaN,03/2012,2012-05-17 12:12:53.0,NaN,03/2012,61


And store the first 10 PCs and subjects age and gender in the covariate file, named ```covariate.txt```

In [66]:
cnames = ["FID", "IID"]
for i in range(1,11):
    cnames.append("PC" + str(i))
cnames += ["AGE", "GENDER"]
covariate.loc[:,cnames].to_csv(
    covariate_fn,
    index=False,
    sep="\t"
)

### Searching SNP-phenotype associations